In [ ]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/he-toxic-multilabel"))

# Any results you write to the current directory are saved as output.
import os
import gc
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from keras import backend as K
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.cross_validation import KFold
from keras.models import Model
from keras.layers import Input, Dense, Embedding, MaxPooling1D, Conv1D, SpatialDropout1D
from keras.layers import add, Dropout, PReLU, BatchNormalization, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import optimizers
from keras import initializers, regularizers, constraints, callbacks
from sklearn.preprocessing import MultiLabelBinarizer
from keras import backend as K


In [ ]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


In [ ]:
train=pd.read_csv('../input/he-toxic-multilabel/d583b256-d-new_dataset/new_dataset/train.csv')
#train=train.sample(frac=.4)
train=train.drop(['id'],axis=1)

train['tags']=train['tags'].astype(str)


train['article']=train['article'].str.replace('</p>|<p>|\r|\n|<br>|</p>|<pre>|</pre>|<code>|</code>','')

train['combined']=train['title']+' '+train['article']

train.drop(['title','article'],axis=1,inplace=True)

lst=[x.split(',') for x in train['tags'].str.replace('|',',').tolist()]


mlb = MultiLabelBinarizer(sparse_output=True)
y=mlb.fit_transform(lst)
del lst

In [ ]:
test=pd.read_csv('../data/he-toxic-multilabel/d583b256-d-new_dataset/new_dataset/test.csv')

test=test.drop(['id'],axis=1)
test['article']=test['article'].str.replace('</p>|<p>|\r|\n|<br>|</p>|<pre>|</pre>|<code>|</code>','')

test['combined']=test['title']+' '+test['article']

test.drop(['title','article'],axis=1,inplace=True)

In [ ]:


def schedule(ind):
    indx=int(ind/8)
    a = [ 0.001, 0.0005, 0.0003, 0.0002,0.0001,0.0001,0.00005]
    return a[indx] 
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint ,ReduceLROnPlateau


class XRAYSequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):

        X_batch = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]

        y_batch = self.y[idx * self.batch_size:(idx + 1) * self.batch_size].todense()

        return np.vstack(X_batch),np.vstack(y_batch)


In [ ]:
EMBEDDING_FILE = '../data/crawl-300d-2M.vec'

X_train = train["combined"].fillna("fillna").values
y_train = y#train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
del y
X_test = test["combined"].fillna("fillna").values




In [ ]:
max_features = 100000
maxlen = 200
embed_size = 300

In [ ]:
del test,train

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
gc.collect()

In [ ]:
del X_train, X_test

In [ ]:
EMBEDDING_FILE = '../data/fasttext-crawl-300d-2m/crawl-300d-2M.vec'
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding="utf8"))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

del all_embs #, X_train, X_test, train, test
gc.collect()

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
print('preprocessing done')

In [ ]:

#model
#wrote out all the blocks instead of looping for simplicity
filter_nr = 64
filter_size = 3
max_pool_size = 3
max_pool_strides = 2
dense_nr = 256
spatial_dropout = 0.2
dense_dropout = 0.5
train_embed = False
conv_kern_reg = regularizers.l2(0.00001)
conv_bias_reg = regularizers.l2(0.00001)

comment = Input(shape=(maxlen,))
emb_comment = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=train_embed)(comment)
emb_comment = SpatialDropout1D(spatial_dropout)(emb_comment)

block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(emb_comment)
block1 = BatchNormalization()(block1)
block1 = PReLU()(block1)
block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block1)
block1 = BatchNormalization()(block1)
block1 = PReLU()(block1)

#we pass embedded comment through conv1d with filter size 1 because it needs to have the same shape as block output
#if you choose filter_nr = embed_size (300 in this case) you don't have to do this part and can add emb_comment directly to block1_output
resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(emb_comment)
resize_emb = PReLU()(resize_emb)
    
block1_output = add([block1, resize_emb])
block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output)

block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block1_output)
block2 = BatchNormalization()(block2)
block2 = PReLU()(block2)
block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block2)
block2 = BatchNormalization()(block2)
block2 = PReLU()(block2)
    
block2_output = add([block2, block1_output])
block2_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output)

block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block2_output)
block3 = BatchNormalization()(block3)
block3 = PReLU()(block3)
block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block3)
block3 = BatchNormalization()(block3)
block3 = PReLU()(block3)
    
block3_output = add([block3, block2_output])
block3_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output)

block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block3_output)
block4 = BatchNormalization()(block4)
block4 = PReLU()(block4)
block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block4)
block4 = BatchNormalization()(block4)
block4 = PReLU()(block4)

block4_output = add([block4, block3_output])
block4_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block4_output)

block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block4_output)
block5 = BatchNormalization()(block5)
block5 = PReLU()(block5)
block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block5)
block5 = BatchNormalization()(block5)
block5 = PReLU()(block5)

block5_output = add([block5, block4_output])
block5_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block5_output)

block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block5_output)
block6 = BatchNormalization()(block6)
block6 = PReLU()(block6)
block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block6)
block6 = BatchNormalization()(block6)
block6 = PReLU()(block6)

block6_output = add([block6, block5_output])
block6_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block6_output)

block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block6_output)
block7 = BatchNormalization()(block7)
block7 = PReLU()(block7)
block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
            kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block7)
block7 = BatchNormalization()(block7)
block7 = PReLU()(block7)

block7_output = add([block7, block6_output])
output = GlobalMaxPooling1D()(block7_output)

output = Dense(dense_nr, activation='linear')(output)
output = BatchNormalization()(output)
output = PReLU()(output)
output = Dropout(dense_dropout)(output)
output = Dense(36321, activation='sigmoid',kernel_initializer=keras.initializers.RandomUniform(minval=-3e-4, maxval=3e-4, seed=None))(output)
#,kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=1, seed=None)
model = Model(comment, output)





In [ ]:

model.compile(loss='binary_crossentropy', 
            optimizer=optimizers.Adam(lr=.001),
            metrics=[fbeta_score])
            

epochs = 40


In [ ]:
batch_size = 512
Xtrain, Xval, ytrain, yval = train_test_split(x_train, y_train, train_size=0.98, random_state=233)



lr = callbacks.LearningRateScheduler(schedule)

xtrain_gen=XRAYSequence(x_train,y_train,batch_size)

eval_gen=XRAYSequence(Xval,yval,batch_size)


In [ ]:
del x_train, y_train

In [ ]:
checkpoint=callbacks.ModelCheckpoint('model_123',save_best_only=True)

In [ ]:

history=model.fit_generator(xtrain_gen,
                        steps_per_epoch=ytrain.shape[0] // batch_size, epochs=epochs,
                       validation_data=eval_gen,
                        validation_steps=yval.shape[0] // batch_size,
                         verbose=1,
                        use_multiprocessing=False,
                            callbacks=[lr]
                       )


In [ ]:
from keras.models import load_model
model.load_weights('model_123')

In [ ]:
test=pd.read_csv('../data/he-toxic-multilabel/d583b256-d-new_dataset/new_dataset/test.csv')['id']

In [ ]:

from keras import backend as K
def difference(vectors):
    x,y=vectors
    return (tf.stop_gradient(y)-x)

def diff_output_shape(shapes):
    shape_x,shape_y=shapes
    return shape_x

def logits(vector):
    import tensorflow as tf
    _epsilon = tf.convert_to_tensor(K.epsilon(), vector.dtype.base_dtype)
    output = tf.clip_by_value(vector, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    #output=output+tf.keras.backend.constant(.4102,vector.dtype.base_dtype)
    return tf.stop_gradient(output)

def logit_output_shape(shape):
    return shape


In [ ]:

from keras.layers import Input, Dense, Lambda, Activation
from keras.models import Model
from keras import backend as K


#input_b=Input()
#input_b=Input(shape=(224,224,3), dtype='float32', name='input')

output_1=Model(inputs=[model.get_input_at(0)],
                                      outputs=[model.layers[63].output])
output_2=Model(inputs=[model.get_input_at(0)],
                                      outputs=[model.layers[65].output])
#x=GlobalAveragePooling2D()(output_1.output)
x = Dense(512)(output_1.output)
x=BatchNormalization()(x)
x= PReLU()(x)
x = Dense(196)(x)
x=BatchNormalization()(x)
x= PReLU()(x)
theta=Dense(36321)(x)

logit=Lambda(logits,output_shape=logit_output_shape,name='logits')(output_2.output)

diff = Lambda(difference,
                  output_shape=diff_output_shape,name='diff')([theta, logit])
diff=Activation('sigmoid')(diff)
model2=Model(inputs=[model.get_input_at(0)],outputs=diff)
model2.summary()


In [ ]:

list_layers=[75,74,73,72,70,69,68,67,66,65,64,]
for layer in range(len(model2.layers)):
    model2.layers[layer].trainable=False
for layer in list_layers:
    model2.layers[layer].trainable=True


In [ ]:
print("training started")

In [ ]:

model2.compile(loss='binary_crossentropy', 
            optimizer=optimizers.Adam(lr=.001),
            metrics=[fbeta_score,precision,recall])
            

epochs = 30


In [ ]:
checkpoint2=callbacks.ModelCheckpoint('model_D',save_best_only=True)

In [ ]:

history3=model2.fit_generator(xtrain_gen,
                        steps_per_epoch=ytrain.shape[0] // batch_size, epochs=epochs,
                       validation_data=eval_gen,
                        validation_steps=yval.shape[0] // batch_size,
                         verbose=1,
                        use_multiprocessing=False,
                            callbacks=[lr,checkpoint2]
                       )


In [ ]:
model2.load_weights('model_D')

In [ ]:

threshold=[.22]
loop=int(194323/batch_size)
remained=194323%batch_size
i=0

for thre in threshold:
    pre=[]
    while True:
        start=i*batch_size
        end=(i+1)*batch_size
        if i==loop:
            end=i*batch_size+remained
            batch_size=remained
            print(start,end)
        pred=model2.predict(x_test[start:end])
        ddx=np.argmax(pred,axis=1)
        pred[np.arange(batch_size),ddx]=1
        pred[pred>=thre]=1
        pred[pred<thre]=0
        tagss=mlb.inverse_transform(pred)
        for tg,idx in zip(tagss,test.values[start:end]):
            strn=''
            for wo in tg:
                strn=strn+wo+'|'
            pre.append({'id':idx,'tags':strn[:-1]})
        i+=1

        if i>loop:
            break


pd.DataFrame(pre).to_csv("submission.csv",index=False)
